# Hospital Patient Claims
## Raw to Staging
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


In [ ]:
# %idle_timeout 10
# %glue_version 5.0
# %worker_type G.1X
# %number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

from pyspark.sql.functions import *
from pyspark.sql.types import * 
from awsglue.dynamicframe import DynamicFrame
import random 
from datetime import datetime
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
todays_date = datetime.today().strftime('%Y-%m-%d')

In [ ]:
s3_path = f's3://hospital-patients-claims-bucket/raw_zone/raw_hospital_patients_claims'

In [ ]:
df = glueContext.create_data_frame.from_options(connection_type='s3', connection_options={"paths": [s3_path]}, format='parquet'
                                                ,push_down_push_down_predicate= f"load_date = '{todays_date}'") 
df = df.cache()

In [ ]:
df = spark.read.csv("data/raw_csv/raw_data_patients_claims.csv", header=True)

In [ ]:
# F.trimming the Columns along with DataType defining
df = df.withColumn('partition_key', F.trim(F.col('partition_key').cast(StringType()))) \
              .withColumn('patient_id', F.trim(F.col('patient_id').cast(StringType()))) \
              .withColumn('name_prefix', F.trim(F.col('name_prefix').cast(StringType()))) \
              .withColumn('first_name', F.trim(F.col('first_name').cast(StringType()))) \
              .withColumn('last_name', F.trim(F.col('last_name').cast(StringType()))) \
              .withColumn('date_of_birth', F.trim(F.col('date_of_birth').cast(StringType()))) \
              .withColumn('phone_number', F.trim(F.col('phone_number').cast(StringType()))) \
              .withColumn('email_id', F.trim(F.col('email_id').cast(StringType()))) \
              .withColumn('policy_id', F.trim(F.col('policy_id').cast(StringType()))) \
              .withColumn('policy_start_date', F.trim(F.col('policy_start_date').cast(StringType()))) \
              .withColumn('policy_end_date', F.trim(F.col('policy_end_date').cast(StringType()))) \
              .withColumn('preimum_amount', F.trim(F.col('preimum_amount').cast(StringType()))) \
              .withColumn('coverage_limit', F.trim(F.col('coverage_limit').cast(StringType()))) \
              .withColumn('addres_id', F.trim(F.col('addres_id').cast(StringType()))) \
              .withColumn('addressline', F.trim(F.col('addressline').cast(StringType()))) \
              .withColumn('borough', F.trim(F.col('borough').cast(StringType()))) \ # Borough
              .withColumn('city', F.trim(F.col('city').cast(StringType()))) \ # London
              .withColumn('state', F.trim(F.col('state').cast(StringType()))) \ # London
              .withColumn('claim_id', F.trim(F.col('claim_id').cast(StringType()))) \
              .withColumn('claim_initialized_date', F.trim(F.col('claim_initialized_date').cast(StringType()))) \
              .withColumn('claim_request_amount', F.trim(F.col('claim_request_amount').cast(StringType()))) \
              .withColumn('load_timestamp', F.trim(F.col('load_timestamp').cast(StringType()))) \
              .withColumn('source_file_name', F.trim(F.col('source_file_name').cast(StringType()))) \
              .withColumn('source_file_path', F.trim(F.col('source_file_path').cast(StringType()))) 

In [ ]:
# Filtering Unwanted Records
df = df.drop_duplicates()
# FIltering by NOT NULL Values on required ID Values
df = df.filter(F.col('Patient_id').isNotNull() & F.col('policy_id').isNotNull() & F.col('addres_id').isNotNull() & F.col('claim_id').isNotNull())


In [ ]:
# Calculate High Risk Claims and add it as a Flag Indicator 
df = df.withColumn('high_risk_claim_flag', F.when(((df.claim_request_amount / df.coverage_limit) * 100) > 80, 'Y').otherwise('N'))


In [ ]:
# Rejected Claims with Rejected Reasons 
df = df.filter(df.claim_status == 'Rejected' & df.claim_reject_reason.isNULL() )

In [ ]:
# Data Enrichment of Borough Details Values
df_borough_details = spark.read.csv('data_london_borough_details.csv')

# Enrich DataFrame with borough abbrevation of London 
combined_df = df.join(F.broadcast(df_borough_details), df.borough = df.borough, 'left')
df = df.withColumn('borough_abbrev', combined_df.borough_abbrev)
	
# Enrich DataFrame with borough code of London 
df = df.withColumn('borough_code', combined_df.borough_code)


In [ ]:
# Process all date-related columns to a standard date/timestamp type
df = df.withColumn('date_of_birth', F.to_date(df.date_of_birth, 'dd-MM-yyyy'))
df = df.withColumn('policy_start_date', F.to_date(df.policy_start_date,'dd/MM/yyyy'))
df = df.withColumn('policy_end_date', F.to_date(df.policy_end_date, 'yyyy/MM/dd'))

# Policy End date should be greater than start date 
df = df.where(df.policy_end_date < df.policy_start_date).show()


In [ ]:
# Checking Amount Column to be having Only Numbers - No Negative
df = df.withColumn('claim_request_amount', F.regexp_replace('claim_request_amount','\$',''))
df = df.withColumn('preimum_amount', F.regexp_replace('preimum_amount','\$',''))
df = df.withColumn('coverage_limit', F.regexp_replace('coverage_limit','\$',''))

# Amount Column Format enforce
df = df.where(df.claim_request_amount > 0)
df = df.where(df.preimum_amount > 0)
df = df.where(df.coverage_limit > 0)

In [ ]:
# INITIAL Caps on Name Fields
df = df.withColumn('first_name', F.initcap(F.col('first_name')))
df = df.withColumn('last_name', F.initcap(F.col('last_name')))

In [ ]:
# Checking Name Column to be having Only Letters
df.withColumn('first_name', regexp_replace(F.col('first_name'), '[^a-zA-Z]', ''))
df.withColumn('last_name', regexp_replace(F.col('last_name'), '[^a-zA-Z]', ''))
df.withColumn('city')

In [ ]:
# Check email format
df = df.filter(F.regexp_like(df.email_id, F.lit(r'^[a-z0-9_.+-]+@[a-z]+\.[a-z]+$')) == True )

In [ ]:
# Check Ph Number
df = df.filter(F.regexp_like('phone_number',F.lit(r'^[0-9]{10}$')) == True) 


In [ ]:
df.write.csv('data/staging_csv/staging_data_patients_claims.csv', header=True)

In [ ]:
def get_secret():
    secret_name = "dev/motor_theft_vehicles/redshift_connection"
    region_name = "us-east-1"
    client = boto3.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    db_config = get_secret_value_response['SecretString']
    return db_config

db_config = json.loads(get_secret())
my_conn_options = {
    "url": db_config['dev_url'],
    "user": db_config['dev_username'],
    "password": db_config['dev_password'],
    "redshiftTmpDir": db_config['dev_redshift_temp_directory'],
}

In [ ]:
dyf = DynamicFrame.fromDF(df)
glueContext.write_dynamic_frame.from_options(
    connection_type="redshift",
    connection_options = my_conn_options,
    frame = dyf_staging_claims,
)